<a href="https://colab.research.google.com/github/DmitryTyurin/data-engineering-public/blob/main/weather_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession